# Spreadsheets on GoogleDocs

- [HFRadar](https://docs.google.com/spreadsheets/d/11hWfIr4lrKP-RviEwSio6dZgnm0oUFUII9WChPDzeAw/edit#gid=62536835)
- [Stations](https://docs.google.com/spreadsheets/d/1ECXoa43uq9Gr8REZF-7EGlxNa1mkP7-4YturX0nFK7s/edit#gid=0)
- [Gliders](https://docs.google.com/spreadsheets/d/13e906UHl6ibF2lx1F_mu_9vBVptVkESM7alw5-FRJdI/edit#gid=65941256)

# Instantiate the map

In [1]:
import folium

lon, lat = -86.276, 30.935 

mapa = folium.Map(location=[lat, lon], zoom_start=5)

## Create the GIS database from the spreadsheets

Change from `False` to `True` to recreate the files.

In [2]:
import os

if True:
    fname = 'hfradar/hfradar.geojson'
    if os.path.isfile(fname):
        os.unlink(fname)

    %pushd hfradar/

    %run convert.py
    %popd

In [3]:
if True:
    fname = 'stations/stations.geojson'
    if os.path.isfile(fname):
        os.unlink(fname)

    %pushd stations/
    %run convert.py
    %popd

/home/filipe/IOOS/SECOORA/static_assets/stations
/home/filipe/IOOS/SECOORA/static_assets
popd -> ~/IOOS/SECOORA/static_assets


In [4]:
if True:
    fname = 'gliders/gliders.geojson'
    if os.path.isfile(fname):
        os.unlink(fname)

    %pushd gliders/
    %run convert.py
    %popd

In [5]:
if True:
    fname = 'glider_tracks/georef_glider_triangles.geojson'
    if os.path.isfile(fname):
        os.unlink(fname)

    fname = 'glider_tracks/gliders_from_georefd.geojson'
    if os.path.isfile(fname):
        os.unlink(fname)

    %pushd glider_tracks/
    %run convert.py
    %popd

In [6]:
if False:
    fname = 'models/models.geojson'
    if os.path.isfile(fname):
        os.unlink(fname)

    %pushd models/
    %run convert.py
    %popd

## Plot Stations/Moorings/Fixed Platforms

In [7]:
import json

with open('stations/stations.geojson') as f:
    stations = json.load(f)


icon_size = (14, 14)
layer_stations = folium.FeatureGroup()
for feature in stations['features']:
    lon, lat = feature['geometry']['coordinates']
    icon_url = feature['properties']['icon']
    popup = feature['properties']['popupcontent']
    
    icon = folium.CustomIcon(icon_url, icon_size=(14, 14))
    
    marker = folium.map.Marker([lat, lon], icon=icon,
                               popup=folium.map.Popup(popup))
    layer_stations.add_children(marker)

layer_stations.layer_name = 'Stations'
mapa.add_children(layer_stations)

## Plot HF-Radar stations and range

In [8]:
with open('hfradar/hfradar.geojson') as f:
    hfradar = json.load(f)

    
style_function = lambda feature: dict(fillColor='#DEFFDE',
                                      color='#AECCAE',
                                      weight=1,
                                      opacity=0.6)

layer_hfradar = folium.FeatureGroup()
for feature in hfradar['features']:
    if feature['geometry']['type'] == 'Point':
        lon, lat = feature['geometry']['coordinates']
        icon_url = feature['properties']['icon']
        popup = feature['properties']['popupcontent']
        
        icon = folium.CustomIcon(icon_url, icon_size=(14, 14))
        marker = folium.map.Marker([lat, lon], icon=icon,
                                   popup=folium.map.Popup(popup))
        layer_hfradar.add_children(marker)
    elif feature['geometry']['type'] == 'Polygon':
        gj = folium.GeoJson(feature, style_function=style_function)
        layer_hfradar.add_children(gj)

layer_hfradar.layer_name = 'HFRadar'
mapa.add_children(layer_hfradar)

## Plot AUVs

In [9]:
import json

with open('gliders/gliders.geojson') as f:
    gliders = json.load(f)


icon_size = (14, 14)
layer_gliders = folium.FeatureGroup()
for feature in gliders['features']:
    lon, lat = feature['geometry']['coordinates']
    icon_url = feature['properties']['icon']
    popup = feature['properties']['popupcontent']
    
    icon = folium.CustomIcon(icon_url, icon_size=(14, 14))
    
    marker = folium.map.Marker([lat, lon], icon=icon,
                               popup=folium.map.Popup(popup))
    layer_gliders.add_children(marker)

layer_gliders.layer_name = 'AUVs'
mapa.add_children(layer_gliders)

## Plot Glider Tracks

In [10]:
with open('glider_tracks/georef_glider_triangles.geojson') as f:
    triangles = json.load(f)

style_function = lambda feature: dict(color='gray', weight=3, opacity=0.6)

layer_tracks = folium.FeatureGroup()
for feature in triangles['features']:
    gj = folium.GeoJson(feature, style_function=style_function)
    layer_tracks.add_children(gj)

In [11]:
with open('glider_tracks/gliders_from_georefd.geojson') as f:
    tracks = json.load(f)


style_function = lambda feature : dict(
    color=feature['properties']['color'],
    weight=2,
    opacity=0.6)

for feature in tracks['features']:
    color = feature['properties']['color']
    gj = folium.GeoJson(feature, style_function=style_function)
    layer_tracks.add_children(gj)

layer_tracks.layer_name = 'Glider Tracks'
mapa.add_children(layer_tracks)

# Plot Numerical Models

In [12]:
with open('models/models.geojson') as f:
    models = json.load(f)

style_function = lambda feature: dict(fillColor='#555555',
                                      color='#555555',
                                      weight=1,
                                      opacity=0.5)

layer_models = folium.FeatureGroup()
for feature in models['features']:
    gj = folium.GeoJson(feature, style_function=style_function)
    layer_models.add_children(gj)

layer_models.layer_name = 'Models'
mapa.add_children(layer_models)

In [13]:
mapa.add_children(folium.LayerControl())

mapa